In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Process data


1.   Unzip both labeled and unlabeled data
2.   Crop Images
3.   Merge provided train and val data into a single dataset
4.   Install packages for pretrained models on ImageNet

External data is from https://dl.allaboutbirds.org/nabirds, 
extract pictures WITHOUT labels with code/preprocess/keep_interesting_classes.py python script

In [ ]:
!mkdir data
!unzip -q 'drive/My Drive/recvis/A3/data/bird_dataset.zip' -d data
!unzip -q 'drive/My Drive/recvis/A3/data/external_bird_dataset.zip' -d data

!rm -rf code
!cp -r 'drive/My Drive/recvis/A3/code' .

!python3 code/preprocess/merge_train_val.py          --src data/bird_dataset                              --dst data/bird_dataset/merged
!python3 code/preprocess/crop_images.py --external 0 --src data/bird_dataset                              --dst data/bird_dataset_cropped.csv
!python3 code/preprocess/crop_images.py --external 1 --src data/external_bird_dataset                     --dst data/external_bird_dataset.csv
!python3 code/preprocess/crop_images.py --external 1 --src data/bird_dataset/test_images/mistery_category --dst data/test_bird_dataset.csv

!python3 code/preprocess/train_val_dataframe.py      --src data/bird_dataset/merged    --dst data/bird_dataset_label.csv

!pip -q install efficientnet_pytorch
!pip -q install pretrainedmodels

import pandas as pd
df_crop = pd.read_csv("data/bird_dataset_cropped.csv") ; df_label = pd.read_csv("data/bird_dataset_label.csv")
df_label.merge(df_crop, on ='Id').to_csv("data/bird_dataset.csv", index=None)

mkdir: cannot create directory ‘data’: File exists
replace data/bird_dataset/test_images/mistery_category/002f61512a368e4c1434eedacf609957.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace data/external_bird_dataset/519.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Train EfficientNet on Fold 0 with low LR

In [ ]:
!python3 code/train.py --k 0 --input_size 300 --freeze 0 --lr 0.0001 --max_lr 0.001 --batch_size 16 --epochs 40 --T1 60 --T2 300 --data_csv data/bird_dataset.csv --experiment experiment --save_best_only 0 --external_data_csv  data/external_bird_dataset.csv

Loaded pretrained weights for efficientnet-b4
Using GPU
Training from epoch=0, until epoch=39 on fold 0
Epoch: 00 / 39 | Alpha Weight: 0.00 | step: 01 | Val acc: 9.24 | Val loss 0.200499
Epoch: 01 / 39 | Alpha Weight: 0.00 | step: 02 | Val acc: 16.81 | Val loss 0.198199
Epoch: 02 / 39 | Alpha Weight: 0.00 | step: 03 | Val acc: 31.09 | Val loss 0.194101
Epoch: 03 / 39 | Alpha Weight: 0.00 | step: 04 | Val acc: 48.74 | Val loss 0.188606
Epoch: 04 / 39 | Alpha Weight: 0.00 | step: 05 | Val acc: 63.87 | Val loss 0.180627
Epoch: 05 / 39 | Alpha Weight: 0.00 | step: 06 | Val acc: 70.59 | Val loss 0.172536
Epoch: 06 / 39 | Alpha Weight: 0.00 | step: 07 | Val acc: 76.47 | Val loss 0.161146
Epoch: 07 / 39 | Alpha Weight: 0.00 | step: 08 | Val acc: 77.31 | Val loss 0.148081
Epoch: 08 / 39 | Alpha Weight: 0.00 | step: 09 | Val acc: 83.19 | Val loss 0.135175
Epoch: 09 / 39 | Alpha Weight: 0.00 | step: 10 | Val acc: 84.03 | Val loss 0.120463
Epoch: 10 / 39 | Alpha Weight: 0.00 | step: 11 | Val acc:

Pick the best models

In [ ]:
!python3 code/keep_best_epoch.py --outfile experiment/best_models.csv --folds 1 --avg 4
import pandas as pd; df = pd.read_csv("experiment/best_models.csv"); print(df.loc[:,'val_acc' ].mean()); print(df.loc[:,'val_acc' ].std())

93.48739624023438
0.420166015625


Test time augmentation: 

* We crop test images using maskrcnn, and retinanet.
* We predict with our checkpoint best models on croped images.
* We repeat with other parameters (threshold, bounding box increased, padding to square images)

In [ ]:
!rm -rf data/bird_dataset
!unzip -q 'drive/My Drive/recvis/A3/data/bird_dataset.zip' -d data
!python3 code/preprocess/save_crop_images.py --src data/test_bird_dataset.csv --threshold 0.4 --inc 0.3 --square 1
!python3 code/predict_best_epoch.py --filename  experiment/best_models.csv --preds_dir experiment/preds --checkpoints experiment/checkpoints 

!rm -rf data/bird_dataset
!unzip -q 'drive/My Drive/recvis/A3/data/bird_dataset.zip' -d data
!python3 code/preprocess/save_crop_images.py --src data/test_bird_dataset.csv --threshold 0.6 --inc 0 --square 0
!python3 code/predict_best_epoch.py --filename  experiment/best_models.csv --preds_dir experiment/preds_v2 --checkpoints experiment/checkpoints 

!rm -rf data/bird_dataset
!unzip -q 'drive/My Drive/recvis/A3/data/bird_dataset.zip' -d data
!python3 code/preprocess/crop_images.py --external 1 --maskrcnn 0 --src data/bird_dataset/test_images/mistery_category --dst data/test_bird_dataset.csv
!python3 code/preprocess/save_crop_images.py --src data/test_bird_dataset.csv --threshold 0.6 --inc 0 --square 1
!python3 code/predict_best_epoch.py --filename  experiment/best_models.csv --preds_dir experiment/preds_v3 --checkpoints experiment/checkpoints 

!cp -f --backup --suffix='.copy' experiment/preds_v2/* experiment/preds/
!cp -f --backup --suffix='.copy.copy' experiment/preds_v3/* experiment/preds/

0it [00:00, ?it/s]Loaded pretrained weights for efficientnet-b4
Using GPU
100% 517/517 [00:18<00:00, 27.60it/s]
Succesfully wrote experiment/preds/preds_0_25, you can upload this file to the kaggle competition website
1it [00:27, 27.06s/it]Loaded pretrained weights for efficientnet-b4
Using GPU
100% 517/517 [00:19<00:00, 27.20it/s]
Succesfully wrote experiment/preds/preds_0_30, you can upload this file to the kaggle competition website
2it [00:54, 27.16s/it]Loaded pretrained weights for efficientnet-b4
Using GPU
100% 517/517 [00:18<00:00, 27.99it/s]
Succesfully wrote experiment/preds/preds_0_35, you can upload this file to the kaggle competition website
3it [01:21, 27.02s/it]Loaded pretrained weights for efficientnet-b4
Using GPU
100% 517/517 [00:18<00:00, 27.88it/s]
Succesfully wrote experiment/preds/preds_0_34, you can upload this file to the kaggle competition website
4it [01:47, 26.96s/it]
0it [00:00, ?it/s]Loaded pretrained weights for efficientnet-b4
Using GPU
100% 517/517 [00:17

Save some spaces by deleting the checkpoints.

Rerun the same process with SE resnext architecture

In [ ]:
!rm -rf experiment/checkpoints
!python3 code/preprocess/merge_train_val.py          --src data/bird_dataset                              --dst data/bird_dataset/merged

In [ ]:
!python3 code/train.py --k 0 --input_size 300 --arch 'se_resnext101_32x4d' --freeze 0 --lr 0.0001 --max_lr 0.001 --batch_size 16 --epochs 40 --T1 60 --T2 300 --data_csv data/bird_dataset.csv --experiment experiment --save_best_only 0 --external_data_csv  data/external_bird_dataset.csv

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext101_32x4d-3b2fe3d8.pth" to /root/.cache/torch/hub/checkpoints/se_resnext101_32x4d-3b2fe3d8.pth
100% 187M/187M [09:12<00:00, 356kB/s]
Using GPU
Training from epoch=0, until epoch=39 on fold 0
Epoch: 00 / 39 | Alpha Weight: 0.00 | step: 01 | Val acc: 6.72 | Val loss 0.198653
Epoch: 01 / 39 | Alpha Weight: 0.00 | step: 02 | Val acc: 21.85 | Val loss 0.191279
Epoch: 02 / 39 | Alpha Weight: 0.00 | step: 03 | Val acc: 47.90 | Val loss 0.178318
Epoch: 03 / 39 | Alpha Weight: 0.00 | step: 04 | Val acc: 74.79 | Val loss 0.150328
Epoch: 04 / 39 | Alpha Weight: 0.00 | step: 05 | Val acc: 81.51 | Val loss 0.107671
Epoch: 05 / 39 | Alpha Weight: 0.00 | step: 06 | Val acc: 84.87 | Val loss 0.078790
Epoch: 06 / 39 | Alpha Weight: 0.00 | step: 07 | Val acc: 87.39 | Val loss 0.068806
Epoch: 07 / 39 | Alpha Weight: 0.00 | step: 08 | Val acc: 87.39 | Val loss 0.063479
Epoch: 08 / 39 | Alpha Weight: 0.00 | step: 09 | Val acc: 89.08 | Val 

In [ ]:
!python3 code/keep_best_epoch.py --outfile experiment/best_models.csv --folds 1 --avg 4
import pandas as pd; df = pd.read_csv("experiment/best_models.csv"); print(df.loc[:,'val_acc' ].mean()); print(df.loc[:,'val_acc' ].std())

93.27731323242188
0.0


In [ ]:
!rm -rf data/bird_dataset
!unzip -q 'drive/My Drive/recvis/A3/data/bird_dataset.zip' -d data
!python3 code/preprocess/save_crop_images.py --src data/test_bird_dataset.csv --threshold 0.2 --inc 0.35 --square 0
!python3 code/predict_best_epoch.py --filename  experiment/best_models.csv --preds_dir experiment/preds_v4 --checkpoints experiment/checkpoints 

!rm -rf data/bird_dataset
!unzip -q 'drive/My Drive/recvis/A3/data/bird_dataset.zip' -d data
!python3 code/preprocess/crop_images.py --external 1 --maskrcnn 1 --src data/bird_dataset/test_images/mistery_category --dst data/test_bird_dataset.csv
!python3 code/preprocess/save_crop_images.py --src data/test_bird_dataset.csv --threshold 0.5 --inc 0.2 --square 1
!python3 code/predict_best_epoch.py --filename  experiment/best_models.csv --preds_dir experiment/preds_v5 --checkpoints experiment/checkpoints 

!rm -rf data/bird_dataset
!unzip -q 'drive/My Drive/recvis/A3/data/bird_dataset.zip' -d data
!python3 code/preprocess/save_crop_images.py --src data/test_bird_dataset.csv --threshold 0.4 --inc 0.1  --square 0
!python3 code/predict_best_epoch.py --filename  experiment/best_models.csv --preds_dir experiment/preds_v6 --checkpoints experiment/checkpoints 

!cp -f --backup --suffix='.copy.copy' experiment/preds_v4/* experiment/preds/
!cp -f --backup --suffix='.copy.copy.copy' experiment/preds_v5/* experiment/preds/
!cp -f --backup --suffix='.copy.copy.copy.copy' experiment/preds_v6/* experiment/preds/

0it [00:00, ?it/s]Using GPU
100% 517/517 [00:23<00:00, 22.23it/s]
Succesfully wrote experiment/preds_v4/preds_0_17, you can upload this file to the kaggle competition website
1it [00:50, 50.14s/it]Using GPU
100% 517/517 [00:23<00:00, 21.59it/s]
Succesfully wrote experiment/preds_v4/preds_0_38, you can upload this file to the kaggle competition website
2it [01:22, 44.95s/it]Using GPU
100% 517/517 [00:23<00:00, 21.95it/s]
Succesfully wrote experiment/preds_v4/preds_0_36, you can upload this file to the kaggle competition website
3it [01:55, 41.21s/it]Using GPU
100% 517/517 [00:23<00:00, 21.73it/s]
Succesfully wrote experiment/preds_v4/preds_0_33, you can upload this file to the kaggle competition website
4it [02:31, 37.94s/it]
0it [00:00, ?it/s]Using GPU
100% 517/517 [00:23<00:00, 22.25it/s]
Succesfully wrote experiment/preds_v5/preds_0_17, you can upload this file to the kaggle competition website
1it [00:32, 32.23s/it]Using GPU
100% 517/517 [00:23<00:00, 22.18it/s]
Succesfully wrote ex

Final submission is average predictions from EfficientNet and ResNext.

In [ ]:
!python3 code/make_sub.py --src experiment/preds --outfile 'drive/My Drive/recvis/A3/sub_v41.csv'